In [38]:
import ee
import os
import time
import logging
import threading
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWjztE4ocJmJkTJFGylYDMLbyd5pOCxhgDUoprO3d-3aXEhad8OWFAU



Successfully saved authorization token.


In [11]:
ee.Initialize()

In [33]:
results_dir = os.path.join(os.getcwd().replace('/code', ''), 'results')

### Import Sentinel-1 collection and watershed basin shapefile

In [13]:
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
basins = ee.FeatureCollection("users/coreyscher/se-asia_subbasin_01min")
lulc = ee.Image('users/coreyscher/all_lcmaps_v3_2018_proj')

In [47]:
classes = {'forest_evergreen': 2,
          'forest_decisuous': 4,
          'forest_mixed': 5,
          'water': 25,
          'urban': 13,
          'barren': 16,
          'grass': 10,
          'savannah' : 9,
          'ag_rice': 17,
          'ag_plantation': 18,
          'ag_pasture_crop': 12,
          'ag_aquaculture': 21,
          'wetland_wetland': 11,
          'wetland_mangrove': 19,
          'wetland_flooded_forest': 20,
          'unclassified': 0}

In [51]:
classes.keys()

dict_keys(['forest_evergreen', 'forest_decisuous', 'forest_mixed', 'water', 'urban', 'barren', 'grass', 'savannah', 'ag_rice', 'ag_plantation', 'ag_pasture_crop', 'ag_aquaculture', 'wetland_wetland', 'wetland_mangrove', 'wetland_flooded_forest', 'unclassified'])

In [14]:
## Set threshold variables 

threshold = -2
vvOwThreshold = -18
vhOwThreshold = -28
connThreshold = 20

## Make a dictionary of dates to loop over

dates = [["2015-01-01", "2016-01-01"],
["2016-01-01", "2017-01-01"],
["2017-01-01", "2018-01-01"],
["2018-01-01", "2019-01-01"],
["2019-01-01", "2020-01-01"]]


In [15]:
## Get a list of basins to iterate over
basins_ids = basins.aggregate_array('ID').getInfo()

In [55]:
# Descending track algorithm

result = None
def run_sub_basin(basin_id, studyStart, studyEnd):
    
    results = []
    
    studyArea = basins.filterMetadata('ID', 'equals', basin_id).first()
    
    # ----Separate S1 by orbit type ---------------------
    s1A = s1.filterBounds(studyArea.geometry())\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.eq('instrumentMode', 'IW')).filterDate(studyStart, studyEnd).filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
    s1D = s1.filterBounds(studyArea.geometry())\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.eq('instrumentMode', 'IW')).filterDate(studyStart, studyEnd).filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    # --------------------------------------------------------------
    
    image_dates = s1D.aggregate_array('system:time_start')
    # Functions
    def medianFilt(image):
        return image.focalMedian(15, 'square', 'meters').clip(studyArea).copyProperties(image)
    ## map median filter 
    s1D = s1D.map(medianFilt)

    def minMax(pol):
        coll = s1D.select([pol])        
        pol = ee.String(pol)
        mn = coll.min().rename([pol.cat('_min')])
        #max = coll.filterDate(lowStart, lowEnd).mean().rename([pol.cat('_max')])

        mx = coll.max().rename([pol.cat('_max')]) #

        mean = coll.mean().rename([pol.cat('_mean')])
        std = coll.reduce(ee.Reducer.stdDev()).rename([pol.cat('_std')]) #.filterDate(lowStart, lowEnd)
        count = coll.count().rename([pol.cat('_count')])
        dif = mx.subtract(mn).rename([pol.cat('_dif')])
        z = mx.subtract(mn).divide(std).rename([pol.cat('_z')])
        return mx.addBands([mn, mean, std, dif, count, z]).set({'pol': pol}).toFloat()
    
    # Calculate relevant statistics for threshold detection 
    vh_stats = minMax('VH')
    vv_stats = minMax('VV')

    vvZMask = vv_stats.select(['VV_z']).gt(2)
    vhZMask = vh_stats.select(['VH_z']).gt(2)
    
    def openWaterClass(image):
        image = ee.Image(image)
        filt = image.focalMedian(15, 'square', 'meters').clip(studyArea).copyProperties(image)
        #vv = image.select(['VV']).lt(vvOwThreshold)
        vh = ee.Image(filt).select(['VH']).lt(vhOwThreshold)


        #vvAreaMask = vv.connectedPixelCount(1024, 'false').gt(connThreshold)
        vhAreaMask = vh.connectedPixelCount(1024, False).gt(connThreshold)

        #vv = vv.updateMask(vv.neq(0)).updateMask(vvZMask).updateMask(vvAreaMask)
        vh = vh.updateMask(vh.neq(0)).updateMask(vhZMask).updateMask(vhAreaMask)

        forest_evergreen = vh.updateMask(lulc.eq(2))
        forest_decisuous = vh.updateMask(lulc.eq(4))
        forest_mixed = vh.updateMask(lulc.eq(5))
        water = vh.updateMask(lulc.eq(25))
        urban = vh.updateMask(lulc.eq(13))
        barren = vh.updateMask(lulc.eq(16))
        grass = vh.updateMask(lulc.eq(10))
        savannah = vh.updateMask(lulc.eq(9))
        ag_rice = vh.updateMask(lulc.eq(17))
        ag_plantation = vh.updateMask(lulc.eq(18))
        ag_pasture_crop = vh.updateMask(lulc.eq(12))
        ag_aquaculture = vh.updateMask(lulc.eq(21))
        wetland_wetland = vh.updateMask(lulc.eq(11))
        wetland_mangrove = vh.updateMask(lulc.eq(19))
        wetland_flooded_forest = vh.updateMask(lulc.eq(20))
        unclassified = vh.updateMask(lulc.eq(0))
        
        
        com = ee.Image(forest_evergreen.copyProperties(image)).set('system:time_start', image.get('system:time_start')) \
                .addBands(forest_decisuous).addBands(forest_mixed).addBands(water).addBands(urban) \
                .addBands(barren).addBands(grass).addBands(savannah).addBands(ag_rice) \
                .addBands(ag_plantation).addBands(ag_pasture_crop).addBands(ag_aquaculture).addBands(wetland_wetland) \
                .addBands(wetland_mangrove).addBands(wetland_flooded_forest).addBands(ag_aquaculture) \
                .rename(['forest_evergreen', 'forest_decisuous', \
                        'forest_mixed', 'water', 'urban', 'barren', \
                        'grass', 'savannah', 'ag_rice', 'ag_plantation', \
                        'ag_pasture_crop', 'ag_aquaculture', 'wetland_wetland', \
                        'wetland_mangrove', 'wetland_flooded_forest', 'unclassified'])

        reduced =com.reduceRegion(ee.Reducer.sum(),
                  geometry=studyArea.geometry(),
                  scale=30)
        date = image.get('system:time_start')
        return reduced
    
    descending = s1D.toList(s1D.size()).map(openWaterClass)
    
    return descending, image_dates

# ls = []
# for d in dates[1:2]:
#     try:
#         studyStart = d[0]
#         studyEnd = d[1]
#         data = run_sub_basin(1958, studyStart, studyEnd)[0].getInfo()
#         i_dates = run_sub_basin(1958,  studyStart, studyEnd)[1].getInfo()
#         data = pd.DataFrame(data)
#         data['dates'] = pd.Series(i_dates)
#         #data.set_index('dates', inplace=True)
#         data
#     except
#         continue

In [56]:
## create a log file

logging.basicConfig(filename="logfilename.log", level=logging.INFO)

In [57]:
def runBasin(basin_id):
    startTime = time.time()
    dest_path = os.path.join(results_dir, 'basin_id_' + str(basin_id) +'.csv')
    # iterate over basin IDs and build a dataframe for each
    data = run_sub_basin(basin_id, '2018-01-01', '2019-01-01')[0].getInfo()
    i_dates = run_sub_basin(basin_id,  '2018-01-01', '2019-01-01')[1].getInfo()
    data_17 = run_sub_basin(basin_id, '2017-01-01', '2018-01-01')[0].getInfo()
    i_dates_17 = run_sub_basin(basin_id,  '2017-01-01', '2018-01-01')[1].getInfo()
    data_16 = run_sub_basin(basin_id, '2016-01-01', '2017-01-01')[0].getInfo()
    i_dates_16 = run_sub_basin(basin_id,  '2016-01-01', '2017-01-01')[1].getInfo()

    df1 = pd.DataFrame(data)
    df1['dates'] = pd.Series(i_dates)
    df2 = pd.DataFrame(data_17)
    df2['dates'] = pd.Series(i_dates_17)
    df3 = pd.DataFrame(data_16)
    df3['dates'] = pd.Series(i_dates_16)

    test_time_series = df3.append(df2).append(df1)
    test_time_series['datetimes'] = pd.to_datetime(test_time_series['dates'], unit='ms')
    test_time_series.set_index('datetimes', inplace=True)
    test_time_series.drop(columns=['dates'], inplace=True)
    
    test_time_series.to_csv(dest_path)
    
    executionTime = (time.time() - startTime)

    logging.info('basin ID '+ str(basin_id)+ ' processed in seconds: ' + str(executionTime))
    
    print('basin ID '+ str(basin_id)+ ' processed in seconds: ' + str(executionTime))
    
    return 'completed basin id: ' + str(basin_id)


In [ ]:
for i in [22, 23]:
    runBasin(i)
    time.sleep(20)

basin ID 22 processed in seconds: 25.408195972442627
